In [1]:
instance_uri = "https://api.stage.sinopia.io/resource/45fe6ec4-81a8-4df3-a75c-1046bf97f3e2"
# change the instance_uri to the URI of the instance that is linked to the work resource you want to serialize for Alma RDF/XML.
# add api key and region for posting the resource to Alma
# https://github.com/LD4P/ils-middleware/wiki/Alma-APIs-in-Airflow

alma_api_key = "alma_api_key"
uri_region = "https://api-na.hosted.exlibrisgroup.com"

In [2]:
%pip install rdflib
%pip install lxml
%pip install requests

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import requests
from rdflib import Graph, Namespace, URIRef
from lxml import etree as ET
from name_space.alma_ns import alma_namespaces
import sys


In [4]:
instance_uri = URIRef(instance_uri)
work_uri = None
instance_graph = Graph()
work_graph = Graph()
instance_graph.parse(instance_uri)

<Graph identifier=Nd7e19d17ec55458294dad1037d82fe8a (<class 'rdflib.graph.Graph'>)>

In [ ]:
# Define the bf and bflc namespaces
bf = Namespace("http://id.loc.gov/ontologies/bibframe/")
bflc = Namespace("http://id.loc.gov/ontologies/bflc/")
# Bind the namespaces to the instance graph
for prefix, url in alma_namespaces:
    instance_graph.bind(prefix, url)
# Get the work URI from the instance graph
work_uri = instance_graph.value(subject=instance_uri, predicate=bf.instanceOf)
# check if the work_uri is none
if work_uri is None:
    print("No work URI found for this instance.")
    sys.exit()

# Ensure work_uri is a URIRef
work_uri = URIRef(work_uri)

# Remove any triples where work_uri is the subject
instance_graph.remove((work_uri, None, None))
# Serialize the instance graph
instance_alma_xml = instance_graph.serialize(format="pretty-xml", encoding="utf-8")

print(instance_alma_xml.decode('utf-8'))

<?xml version="1.0" encoding="utf-8"?>
<rdf:RDF
  xmlns:bf="http://id.loc.gov/ontologies/bibframe/"
  xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
  xmlns:sinopiabf="http://sinopia.io/vocabulary/bf/"
  xmlns:bflc="http://id.loc.gov/ontologies/bflc/"
  xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#"
  xmlns:sinopia="http://sinopia.io/vocabulary/"
>
  <bf:Instance rdf:about="https://api.stage.sinopia.io/resource/45fe6ec4-81a8-4df3-a75c-1046bf97f3e2">
    <bf:note>
      <bf:Note rdf:nodeID="f5de59cc025b644beaf93b0a0991327b6b6">
        <rdfs:label xml:lang="en">"A Robbins Publication"</rdfs:label>
      </bf:Note>
    </bf:note>
    <bf:note>
      <bf:Note rdf:nodeID="f5de59cc025b644beaf93b0a0991327b6b7">
        <rdfs:label xml:lang="en">BF test. Do not use for Acquisitions or Cataloging</rdfs:label>
      </bf:Note>
    </bf:note>
    <rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/Print"/>
    <sinopia:hasResourceTemplate>pcc:bf2:NotatedMusic:Instance</sin

In [6]:
tree = ET.fromstring(instance_alma_xml)
# apply xslt to normalize instance
xslt = ET.parse("xsl/normalize-instance-sinopia2alma.xsl")
transform = ET.XSLT(xslt)
instance_alma_xml = transform(tree)
instance_alma_xml = ET.tostring(
        instance_alma_xml, pretty_print=True, encoding="utf-8"
        )
# save the xml to a file
with open("alma-instance.xml", "wb") as f:
    f.write(instance_alma_xml)
print(instance_alma_xml.decode("utf-8"))

<bib>
  <record_format>lcbf_instance</record_format>
  <suppress_from_publishing>false</suppress_from_publishing>
  <record>
    <rdf:RDF xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#" xmlns:bf="http://id.loc.gov/ontologies/bibframe/" xmlns:bflc="http://id.loc.gov/ontologies/bflc/" xmlns:sinopia="http://sinopia.io/vocabulary/" xmlns:owl="http://www.w3.org/2002/07/owl#" xmlns:skos="http://www.w3.org/2004/02/skos/core#" xmlns:dcterms="http://purl.org/dc/terms/" xmlns:cc="http://creativecommons.org/ns#" xmlns:foaf="http://xmlns.com/foaf/0.1/" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">
  <bf:Instance rdf:about="https://api.stage.sinopia.io/resource/45fe6ec4-81a8-4df3-a75c-1046bf97f3e2" xmlns:bf="http://id.loc.gov/ontologies/bibframe/">
    <bf:note>
      <bf:Note rdf:nodeID="f5de59cc025b644beaf93b0a0991327b6b6">
        <rdfs:label xml:lang="en">"A Robbins Publication"</rdfs:label>
      </bf:Note>
    </bf:note>
    <bf:note>
      <bf:Note rdf:nodeID="f5de59cc025b644be

In [7]:
# handle 400, an update to the record in Alma
def parse_400(result):
    xml_response = ET.fromstring(result)
    xslt = ET.parse("xsl/put_mms_id.xsl")
    transform = ET.XSLT(xslt)
    result_tree = transform(xml_response)
    put_mms_id_str = str(result_tree)
    print(f"put_mms_id_str: {put_mms_id_str}")
    return put_mms_id_str

# post the instance to Alma
def NewInstancetoAlma():
    with open("alma-instance.xml", "rb") as f:
        data = f.read()

        alma_uri = (
            uri_region
            + "/almaws/v1/bibs?"
            + "from_nz_mms_id=&from_cz_mms_id=&normalization=&validate=false"
            + "&override_warning=true&check_match=false&import_profile=&apikey="
            + alma_api_key
        )
        # post to alma
        alma_result = requests.post(
            alma_uri,
            headers={
                "Content-Type": "application/xml; charset=utf-8",
                "Accept": "application/xml",
                "x-api-key": alma_api_key,
            },
            data=data,
        )
        print(f"alma result: {alma_result.status_code}\n{alma_result.text}")
        result = alma_result.content
        status = alma_result.status_code
        if status == 200:
            xml_response = ET.fromstring(result)
            mms_id = xml_response.xpath("//mms_id/text()")
            print(f"Created record {mms_id}")
        elif status == 400:
            # run xslt on the result in case the response is 400 and we need to update the record
            put_mms_id_str = parse_400(result)
            alma_update_uri = (
                uri_region
                + "/almaws/v1/bibs/"
                + put_mms_id_str
                + "?normalization=&validate=false&override_warning=true"
                + "&override_lock=false&stale_version_check=false&cataloger_level=&check_match=false"
                + "&import_profile=&apikey="
                + alma_api_key
            )
            putInstanceToAlma(
                alma_update_uri,
                data,
            )
        else:
            raise Exception(f"Unexpected status code from Alma API: {status}")

# update the instance in Alma
def putInstanceToAlma(
    alma_update_uri,
    data,
):
    put_update = requests.put(
        alma_update_uri,
        headers={
            "Content-Type": "application/xml; charset=UTF-8",
            "Accept": "application/xml",
        },
        data=data,
    )
    print(f"put update: {put_update.status_code}\n{put_update.text}")
    put_update_status = put_update.status_code
    result = put_update.content
    xml_response = ET.fromstring(result)
    put_mms_id = xml_response.xpath("//mms_id/text()")
    match put_update_status:
        case 200:
            print(f"Updated record {put_mms_id}")
        case 500:
            raise Exception(f"Internal server error from Alma API: {put_update_status}")
        case _:
            raise Exception(
                f"Unexpected status code from Alma API: {put_update_status}"
            )

# Call the function
NewInstancetoAlma()

alma result: 400
<?xml version="1.0" encoding="UTF-8" standalone="yes"?><web_service_result xmlns="http://com/exlibris/urm/general/xmlbeans"><errorsExist>true</errorsExist><errorList><error><errorCode>60307</errorCode><errorMessage>Matched instance records exist in the catalog with ID: [9979506124303681]</errorMessage><trackingId>E01-0411220319-X9ZM0-AWAE441094233</trackingId></error></errorList></web_service_result>
put_mms_id_str: 9979506124303681
put update: 200
<?xml version="1.0" encoding="UTF-8" standalone="yes"?><bib><mms_id>9979506124303681</mms_id><record_format>lc_bf_instance</record_format><linked_record_id/><title>Meditation a novelty piano solo</title><author>Sims, Lee</author><network_numbers><network_number>(ocolc)53068061</network_number></network_numbers><place_of_publication>New York</place_of_publication><date_of_publication>[1927]</date_of_publication><publisher_const>Robbins Music Corporation</publisher_const><holdings link="https://api-na.hosted.exlibrisgroup.com/